<a href="https://colab.research.google.com/github/BorgesFe/curso_agentesia/blob/main/C%C3%B3pia_de_Agente_grupo_GTIA_(vers%C3%A3o_1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Agente GTIA**

Este notebook é um trabalho do curso Agentes Autônomos com Redes Generativas - I2A2.

Ele permite que você pergunte qualquer coisa sobre os dados nos arquivos CSV e o agente vai procurar diretamente a resposta neles.


In [21]:
# Instalação de bibliotecas - organização do ambiente de trabalho
!pip install -q openai pandas python-dotenv gradio

import os
import zipfile
import pandas as pd
from openai import OpenAI
import gradio as gr
import io

In [22]:
# Montagem do Google Drive e leitura da chave .env

try:
    from google.colab import drive
    drive.mount('/content/drive')

    zip_path = "/content/drive/MyDrive/202401--NFs.zip"

    # Leitura do arquivo .env com a chave da OpenAI
    from dotenv import load_dotenv
    import os

    env_path = '/content/drive/MyDrive/segredos/.env'
    load_dotenv(dotenv_path=env_path)

    os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")



except ImportError:
    print("Google Colab drive não está disponível. Certifique-se de que o arquivo ZIP está no caminho correto para execução local.")
    zip_path = "202401--NFs.zip"

extract_path = "/content/nfs"  # Pasta onde os CSVs serão extraídos

# Garante que o diretório de extração exista
os.makedirs(extract_path, exist_ok=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:

# Extração do ZIP contendo os CSVs
try:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print(f"Arquivos extraídos para: {extract_path}")
except FileNotFoundError:
    print(f"Erro: Arquivo ZIP não encontrado em {zip_path}. Por favor, verifique o caminho.")
    exit()
except Exception as e:
    print(f"Erro ao extrair o arquivo ZIP: {e}")
    exit()


Arquivos extraídos para: /content/nfs


In [24]:
# Carregamento dos dados
try:
    cabecalho_df = pd.read_csv(f"{extract_path}/202401_NFs_Cabecalho.csv")
    itens_df = pd.read_csv(f"{extract_path}/202401_NFs_Itens.csv")
    print("Dados CSV carregados com sucesso.")

    # Mostrar informações dos dados
    print(f"\nCabeçalho - {len(cabecalho_df)} registros")
    print("Colunas:", list(cabecalho_df.columns))

    print(f"\nItens - {len(itens_df)} registros")
    print("Colunas:", list(itens_df.columns))

except Exception as e:
    print(f"Erro ao carregar os arquivos CSV: {e}")
    cabecalho_df = pd.DataFrame()
    itens_df = pd.DataFrame()

def perguntar_ao_agente(pergunta):
    """
    Agente que usa OpenAI para interpretar perguntas e gerar análises dos dados
    """

    if not pergunta.strip():
        return "Por favor, faça uma pergunta sobre os dados das notas fiscais."

    # Verifica se os dados foram carregados
    if cabecalho_df.empty or itens_df.empty:
        return "Erro: Dados não carregados. Verifique os arquivos CSV."

    # Cria contexto sobre os dados disponíveis
    contexto_dados = f"""
    DADOS DISPONÍVEIS:

    1. CABEÇALHO DAS NOTAS FISCAIS ({len(cabecalho_df)} registros):
    Colunas: {', '.join(cabecalho_df.columns)}

    2. ITENS DAS NOTAS FISCAIS ({len(itens_df)} registros):
    Colunas: {', '.join(itens_df.columns)}

    PERGUNTA DO USUÁRIO: {pergunta}

    Por favor, analise a pergunta e gere código Python usando pandas para respondê-la.
    Use as variáveis 'cabecalho_df' e 'itens_df'.

    IMPORTANTE:
    - Retorne apenas o código Python executável
    - Use try/except para tratamento de erros
    - Armazene o resultado final em uma variável chamada 'resposta'
    - Se for um DataFrame grande, limite a 10 registros
    - Formate valores monetários adequadamente

    Exemplo:
    try:
        resultado = itens_df.groupby('coluna_fornecedor')['coluna_valor'].sum()
        maior_fornecedor = resultado.idxmax()
        maior_valor = resultado.max()
        resposta = f"Maior fornecedor: {{maior_fornecedor}} com R$ {{maior_valor:,.2f}}"
    except Exception as e:
        resposta = f"Erro na análise: {{str(e)}}"
    """

    try:
        # Chama a OpenAI para gerar o código
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{
                "role": "system",
                "content": "Você é um especialista em análise de dados com pandas. Gere código Python limpo e executável."
            }, {
                "role": "user",
                "content": contexto_dados
            }],
            temperature=0.1,
            max_tokens=500
        )

        codigo_gerado = response.choices[0].message.content.strip()

        # Remove formatação markdown se presente
        if "```python" in codigo_gerado:
            codigo_gerado = codigo_gerado.split("```python")[1].split("```")[0].strip()
        elif "```" in codigo_gerado:
            codigo_gerado = codigo_gerado.split("```")[1].strip()

        # Executa o código gerado em ambiente seguro
        ambiente_execucao = {
            'cabecalho_df': cabecalho_df,
            'itens_df': itens_df,
            'pd': pd,
            'resposta': None
        }

        exec(codigo_gerado, {"__builtins__": {}}, ambiente_execucao)

        resultado = ambiente_execucao.get('resposta')

        if resultado is None:
            return "Não foi possível gerar uma resposta para sua pergunta."

        return str(resultado)

    except Exception as e:
        # Fallback: tenta responder com lógica pré-definida
        return responder_perguntas_comuns(pergunta)

def responder_perguntas_comuns(pergunta):
    """
    Fallback para perguntas comuns quando a IA falha
    """
    pergunta_lower = pergunta.lower()

    try:
        # Identifica possíveis colunas de fornecedor e valor
        colunas_fornecedor = [col for col in itens_df.columns if any(word in col.lower() for word in ['fornecedor', 'empresa', 'razao'])]
        colunas_valor = [col for col in itens_df.columns if any(word in col.lower() for word in ['valor', 'preco', 'total'])]
        colunas_quantidade = [col for col in itens_df.columns if any(word in col.lower() for word in ['qtd', 'quantidade', 'volume'])]
        colunas_item = [col for col in itens_df.columns if any(word in col.lower() for word in ['item', 'produto', 'descricao'])]

        if "fornecedor" in pergunta_lower and "maior" in pergunta_lower and "montante" in pergunta_lower:
            if colunas_fornecedor and colunas_valor:
                col_fornecedor = colunas_fornecedor[0]
                col_valor = colunas_valor[0]

                grupo = itens_df.groupby(col_fornecedor)[col_valor].sum()
                fornecedor_maior = grupo.idxmax()
                valor_maior = grupo.max()

                return f"Fornecedor com maior montante: {fornecedor_maior}\nValor total: R$ {valor_maior:,.2f}"

        elif "item" in pergunta_lower and "maior" in pergunta_lower and "volume" in pergunta_lower:
            if colunas_item and colunas_quantidade:
                col_item = colunas_item[0]
                col_qtd = colunas_quantidade[0]

                grupo = itens_df.groupby(col_item)[col_qtd].sum()
                item_maior = grupo.idxmax()
                qtd_maior = grupo.max()

                return f"Item com maior volume: {item_maior}\nQuantidade total: {qtd_maior}"

        elif "total" in pergunta_lower and "notas" in pergunta_lower:
            return f"Total de notas fiscais: {len(cabecalho_df)}"

        elif "valor" in pergunta_lower and "total" in pergunta_lower:
            if colunas_valor:
                col_valor = colunas_valor[0]
                valor_total = itens_df[col_valor].sum()
                return f"Valor total de todas as notas: R$ {valor_total:,.2f}"

        else:
            # Lista as colunas disponíveis para ajudar o usuário
            return f"""
            Não consegui entender sua pergunta.

            Dados disponíveis:

            CABEÇALHO ({len(cabecalho_df)} registros):
            {', '.join(cabecalho_df.columns)}

            ITENS ({len(itens_df)} registros):
            {', '.join(itens_df.columns)}

            Tente perguntas como:
            - Qual fornecedor teve maior montante recebido?
            - Qual item teve maior volume entregue?
            - Quantas notas fiscais temos no total?
            - Qual o valor total de todas as notas?
            """

    except Exception as e:
        return f"Erro ao processar a pergunta: {str(e)}"

# Lista de exemplos
exemplos_perguntas = [
    "Qual o fornecedor que teve maior montante recebido?",
    "Qual item teve maior volume entregue?",
    "Quantas notas fiscais temos no total?",
    "Qual o valor total de todas as notas fiscais?",
    "Quais são os 5 maiores valores de itens?"
]

Dados CSV carregados com sucesso.

Cabeçalho - 100 registros
Colunas: ['CHAVE DE ACESSO', 'MODELO', 'SÉRIE', 'NÚMERO', 'NATUREZA DA OPERAÇÃO', 'DATA EMISSÃO', 'EVENTO MAIS RECENTE', 'DATA/HORA EVENTO MAIS RECENTE', 'CPF/CNPJ Emitente', 'RAZÃO SOCIAL EMITENTE', 'INSCRIÇÃO ESTADUAL EMITENTE', 'UF EMITENTE', 'MUNICÍPIO EMITENTE', 'CNPJ DESTINATÁRIO', 'NOME DESTINATÁRIO', 'UF DESTINATÁRIO', 'INDICADOR IE DESTINATÁRIO', 'DESTINO DA OPERAÇÃO', 'CONSUMIDOR FINAL', 'PRESENÇA DO COMPRADOR', 'VALOR NOTA FISCAL']

Itens - 565 registros
Colunas: ['CHAVE DE ACESSO', 'MODELO', 'SÉRIE', 'NÚMERO', 'NATUREZA DA OPERAÇÃO', 'DATA EMISSÃO', 'CPF/CNPJ Emitente', 'RAZÃO SOCIAL EMITENTE', 'INSCRIÇÃO ESTADUAL EMITENTE', 'UF EMITENTE', 'MUNICÍPIO EMITENTE', 'CNPJ DESTINATÁRIO', 'NOME DESTINATÁRIO', 'UF DESTINATÁRIO', 'INDICADOR IE DESTINATÁRIO', 'DESTINO DA OPERAÇÃO', 'CONSUMIDOR FINAL', 'PRESENÇA DO COMPRADOR', 'NÚMERO PRODUTO', 'DESCRIÇÃO DO PRODUTO/SERVIÇO', 'CÓDIGO NCM/SH', 'NCM/SH (TIPO DE PRODUTO)', 'CFO

In [25]:
# Criação da interface Gradio
import gradio as gr

iface = gr.Interface(
    fn=perguntar_ao_agente,
    inputs=gr.Textbox(lines=2, placeholder="Digite sua pergunta sobre os dados das Notas Fiscais aqui..."),
    outputs="text",
    title="GTIA - Pesquise nas NFs",
    description=(
        "Faça sua pergunta!"
    ),
    live=False, # Não atualiza em tempo real enquanto o usuário digita
    css="""
    body { font-family: 'Inter', sans-serif; background-color: #f0f2f5; }
    .gradio-container { max-width: 800px; margin: 40px auto; border-radius: 15px; overflow: hidden; box-shadow: 0 10px 30px rgba(0,0,0,0.1); }
    .gr-header { background-color: #4CAF50; color: white; padding: 20px; text-align: center; border-radius: 15px 15px 0 0; }
    .gr-interface-panel { padding: 20px; }
    .gr-textbox label { font-weight: bold; color: #333; }
    .gr-textbox textarea { border-radius: 8px; border: 1px solid #ddd; padding: 10px; transition: all 0.3s ease; }
    .gr-textbox textarea:focus { border-color: #4CAF50; box-shadow: 0 0 5px rgba(76, 175, 80, 0.5); outline: none; }
    .gr-button { background-color: #007bff; color: white; border: none; border-radius: 8px; padding: 10px 20px; font-size: 16px; cursor: pointer; transition: background-color 0.3s ease, transform 0.2s ease; }
    .gr-button:hover { background-color: #0056b3; transform: translateY(-2px); }
    .gradio-output { background-color: #e9ecef; border-radius: 8px; padding: 15px; border: 1px solid #ced4da; margin-top: 20px; white-space: pre-wrap; word-wrap: break-word; }
    """
)

# Lançamento da interface Gradio
if __name__ == "__main__":
    print("Iniciando a interface Gradio...")
    iface.launch(share=True) # share=True gera um link público temporário para compartilhar

Iniciando a interface Gradio...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a8e9050bdb6e9a588a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
